In [ ]:
# -*- coding: utf-8 -*-
#  Copyright 2021 - 2022 United Kingdom Research and Innovation
#  Copyright 2021 - 2022 The University of Manchester
#  Copyright 2021 - 2022 Technical University of Denmark
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#   Authored by:    Edoardo Pasca (UKRI-STFC)


# Tomographic reconstruction with Least Squares and Total Generalised Variation regularisation

### Dataset (probably the ICASSP24 challenge?)

This exercise walks through the steps needed to load in and reconstruct by FDK a 3D cone-beam dataset of a walnut, acquired by laboratory micro-CT.

Learning objectives are:
- Set up an optimisation problem to solve Least Squares data fitting with Total Generalised Variation regularisation
- Solve the problem with the PDHG algorithm

This example requires the dataset from https://zenodo.org/records/8377374 :

If running locally please download the data and update the 'path' variable below.


## Total Generalised Variation

For an introduction on TGV, refer to [Core Imaging Library part II](http://doi.org/10.1098/rsta.2020.0193) section 3b.

$$
u^* = \arg\min_u \frac{1}{2} \| A u - b \|_2^2 + TGV_{\alpha, \delta}(u)
$$

This can be written as 

$$
u^*, w^* = \arg\min_{u, w} \frac{1}{2} \| A u - b \|_2^2 + \alpha  \left( \| \nabla u - w \|_{2,1} + \delta \| \mathcal{E} w \|_{2,1} \right)
$$

where $\mathcal{E}$ is the symmetrised gradient operator, `SymmetrisedGradientOperator` in `ccpi.optimisation.operators`. $u$ is the image, $w$ is the gradient auxiliary variable and $\alpha$ and $\delta$ are the regularisation parameters. Notice that the second term of the regularisation function is scaled
by $\delta$ in order to provide a way of explaining this to the user.


In [ ]:
path = '/mnt/materials/SIRF/Fully3D/CIL/Walnut'

In [ ]:
# remove some annoying warnings
import logging
logger = logging.getLogger('dxchange')
logger.setLevel(logging.ERROR)

First import all of the modules we will need:

In [ ]:
import os

from cil.optimisation.functions import MixedL21Norm, BlockFunction, L2NormSquared
from cil.optimisation.operators import BlockOperator, IdentityOperator, GradientOperator, \
    SymmetrisedGradientOperator, ZeroOperator

from cil.utilities.display import show2D, show_geometry
from cil.utilities.jupyter import islicer, link_islicer

from cil.framework import ImageGeometry, AcquisitionGeometry, DataOrder
from cil.utilities.display import show2D, show_geometry, show1D
from cil.framework import AcquisitionData
from cil.recon import FDK
from cil.plugins.tigre import ProjectionOperator
from cil.utilities.quality_measures import mse
from cil.framework import ImageData
from cil.optimisation.functions import ZeroFunction
from cil.optimisation.algorithms import PDHG

from cil.processors import Padder

import numpy as np


Load the data by setting the `AcquisitionGeometry` as described in the dataset documentation.

In [ ]:
#%%
print (data.shape)
print (DataOrder.ASTRA_AG_LABELS)
#%%
image_size = [300, 300, 300]
image_shape = [256, 256, 256]
voxel_size = [1.171875, 1.171875, 1.171875]

detector_shape = [256, 256]
detector_size = [600, 600]
pixel_size = [2.34375, 2.34375]

distance_source_origin = 575
distance_source_detector = 1050

#  S---------O------D
#

angles = np.linspace(0, 2*np.pi, 360, endpoint=False)
#%%
AG = AcquisitionGeometry.create_Cone3D(source_position=[0, -distance_source_origin, 0],\
                                       detector_position=[0, distance_source_detector-distance_source_origin, 0],)\
                                        .set_angles(-angles, angle_unit='radian')\
                                        .set_panel(detector_shape, pixel_size, origin='bottom-left')\
                                        .set_labels(DataOrder.ASTRA_AG_LABELS[:])
ig = ImageGeometry(voxel_num_x=image_shape[0], voxel_num_y=image_shape[1], voxel_num_z=image_shape[2],\
                     voxel_size_x=voxel_size[0], voxel_size_y=voxel_size[1], voxel_size_z=voxel_size[2])
#%%


Visualise the geometries. Are they correct?

In [ ]:

show_geometry(AG, ig)


Load the data into a numpy array and then into an `AcquisitionData` object with the appropriate `AcquisitionGeometry`.

In [ ]:
dose = 'low'
data_directory = '/opt/data/ICASSP24/train/train/'
filename = os.path.join(data_directory, f'0000_sino_{dose}_dose.npy')

data=np.asarray(np.load(filename,allow_pickle=True), dtype=np.float32)

ad = AcquisitionData(data, geometry=AG)

#%%


Pad the acquisition data to avoid artefacts in the reconstruction.

In [ ]:

# 256 + 2*pad=256*sqrt(2)
pad = np.ceil( 256*(np.sqrt(2)-1)/2 ).astype(np.int32)
ad = Padder(pad_width={'horizontal':100})(ad)
#%%
ad.reorder('tigre')

Let's try in 2D first

In [ ]:
data2d = ad.get_slice(vertical='centre')
ig2d = ig.get_slice(vertical='centre')

To set up the optimisation problem, let's first define the projection operator $A$.

In [ ]:
# Define ProjectionOperator A
A = ProjectionOperator(ig2d, data2d.geometry)


### regularisation parameters

In [ ]:
alpha = 100.
gamma = 1.0

We are ready to construct the block operator. Remember that the rows of the `BlockOperator` have the same range, while the columns have the same domain.

In [ ]:
beta = alpha / gamma
    

# Define BlockOperator K
            
# Set up the 3 operator A, Grad and Epsilon                           
# A, the projection operator is passed by the user    
K11 = A
K21 = alpha * GradientOperator(K11.domain)
# https://tomographicimaging.github.io/CIL/nightly/optimisation.html#cil.optimisation.operators.SymmetrisedGradientOperator
K32 = beta * SymmetrisedGradientOperator(K21.range)
# these define the domain and range of the other operators
K12 = ZeroOperator(K32.domain, K11.range)
K22 = -alpha * IdentityOperator(domain_geometry=K21.range, range_geometry=K32.range)
K31 = ZeroOperator(K11.domain, K32.range)

K = BlockOperator(K11, K12, K21, K22, K31, K32, shape=(3,2) )

Let's define the data fidelity term. We will use the `LeastSquares` class and the norms for the TGV terms. We will block them in a `BlockFunction` object.

In [ ]:

f1 = 0.5 * L2NormSquared(b=data)
f2 = MixedL21Norm()
f3 = MixedL21Norm() 
F = BlockFunction(f1, f2, f3)         

It's almost all set to initialise the PDHG algorithm. We can add an additional constraint in the function $g$ accepted by PDHG. For this we will start with no constraint: `ZeroFunction`. Another option is the `IndicatorBox` for upper and lower bounds.

In [ ]:
g = ZeroFunction()

It's time to initialise the PDHG algorithm.

In [ ]:
algo = PDHG(f=F, g=g, operator=K, max_iteration=2000, update_objective_interval=100)

### Find the right step sizes
Need to do a parameter search for the step sizes. We will use the `gammad` parameter. Try setting it very small and very large and see how the algorithm behaves. You should find a value that makes the algorithm converge in a reasonable number of iterations.

In [ ]:

gammad = 11
# sigma/tau = gammad
 
tau = 1 /  ( K.norm() * gammad )
sigma = gammad / K.norm()

algo.set_step_sizes(sigma=sigma, tau=tau)

#%%
# RUN PDHG
algo.run(1000,verbose=2)